## Shoreline extraction from Landsat MNDWI images for Northern Alaska (1986-2021)

This notebook extracts subpixel contours from MNDWI images, which have been processing on  Google Earth Engine (see: https://code.earthengine.google.com/9d450b3d0d9840a352cbd3ba6ddb6ca5) and quantifies coastline change along shore-perpendicular transects.

Content of the notebook:

1. Setup
2. Download MNDWI rasters from Google Drive
3. Reproject and quality check MNDWI rasters
4. Subpixel contours
5. Create minimum water extent polygon
6. Create transects
7. Calculate intersections between shorelines and transects
8. Coastline Change Analysis


Further notes on Alaska: 
+ Projection: (EPSG 3413 (WGS 84/ NSIDC Sea Ice Polar Stereographic North), projected coordinate system for polar research)
+ Projection: EPSG 5936 (WGS 84/ Alaska Polar Stereographic), projected coordinate system 
+ Polar night: from Novermber 18 or 19 for 66 days
+ Cloud cover: "Utqiavik is one of the cloudiest places on Earth", completely overcast for more than 50% of the day. 70% overcast for 62% of the time. Dense fog on 65 days per year in the summer months. (Maykut, Gary A.; Church, Phil E. (1973). Journal of Applied Meteorology. Department of Atmospheric Sciences, University of Washington. pp. 620–621.)
+ Snow: "Freezing and snowfall an occur during any month of the year" (https://web.archive.org/web/20130119021827/http://climate.gi.alaska.edu/Stations/Arctic/Barrow.html)

Interesting data: 
+ VHS Alaska (50 cm): https://soa-dnr.maps.arcgis.com/home/webmap/viewer.html?webmap=13dd1ccf165845eea5db36465e7d565c
+ 

### 1.| Setup

In [1]:
# Libraries
import os
import glob 
import numpy as np 
import shapely as shp
import pandas as pd 
import geopandas as gpd 
import rasterio as rio 
import matplotlib.pyplot as plt
from scipy import stats 
from skimage.filters import threshold_otsu
# my coastline methods 
from coasty import postprocess, analysis 

In [2]:
# Directories
data_dir = os.path.join(os.getcwd(),"data/Alaska_north") # path to data-folder with aux data
plot_dir = os.path.join(os.getcwd(),"figures/plots")
country_bounds_path = os.path.join(data_dir,"AK_north_slope_bounds.geojson") # path to country bounds
transects_path = os.path.join(data_dir,"AKn_transects_gov_2km_200m")
osm_sl_path = os.path.join(os.path.join(data_dir,"AKn_osm_coastline.geojson")) # path to reference shoreline
buffer_path = os.path.join(os.path.join(data_dir,"AKn_buffer_5km"))
tile_name = "Barrow"

# Params
export_folder = "GEE_alaska"       # folder on Google Drive with GEE images to download
crs = "EPSG:5936"                   # coordinate system code of a projected crs 
min_length = 3000                   # min length of shoreline to keep [m]
buffer_dist = 5000                  # buffer around reference shorelines to clip detected shorelines [m]
transect_len = 3000                 # length of transects [m]
transect_dist = 200                 # distance between transects [m]
transect_min_line_length = 10000    # min legnth of polygon outline at which to draw transects [m]
                                    # (for removing small islands) 

In [8]:
# read/ create aux data 
#country_bounds = gpd.read_file(country_bounds_path).to_crs(crs)
osm_sl = gpd.read_file(osm_sl_path).to_crs(crs)

try:
    buffer = gpd.read_file(buffer_path)
    print("Everything successfully read.")
except:
    print("Create osm shoreline buffer:")
    buffer = osm_sl.buffer(buffer_dist)
    buffer.to_file(buffer_path,driver="GeoJSON")
    print("Buffer saved.")

Everything successfully read.


### 2.| Download MNDWI rasters from Google Drive
done manually in this case. 

### 3.| Reproject and quality check MNDWI rasters

In [11]:
print(("--")*10,"Treating",tile_name,("--")*10)
folder_path = os.path.join(data_dir,tile_name)
raster_paths = glob.glob(os.path.join(data_dir,tile_name,tile_name+"_*.tif"))
raster_paths.sort()
for r in raster_paths:
    masked_file = glob.glob(r[:-4]+"*aq.tif") 
    if len(masked_file) == 0:
        postprocess.reproject_raster(r,r,crs)
        try:
            postprocess.mask_single_observation_pixel(r)
            os.remove(r)
            print(os.path.basename(r),"removed.")
        except:
            print(os.path.basename(r),'could not be masked')
            pass
    else:
        print(os.path.basename(masked_file[0]), "already exists.")

-------------------- Treating Barrow --------------------
Barrow_1985.tif reprojected.
Barrow_1985.tif masked.
Barrow_1985_01avg_aq.tif saved.
Barrow_1985.tif removed.
Barrow_1986.tif reprojected.
Barrow_1986.tif masked.
Barrow_1986_04avg_aq.tif saved.
Barrow_1986.tif removed.
Barrow_1987.tif reprojected.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/coasty/postprocess.py:86: RuntimeWarning: Mean of empty slice
  avg_aq = int(np.nanmean(nobs))
Barrow_1987.tif could not be masked
Barrow_1990.tif reprojected.
Barrow_1990.tif masked.
Barrow_1990_01avg_aq.tif saved.
Barrow_1990.tif removed.
Barrow_1991.tif reprojected.
Barrow_1991.tif could not be masked
Barrow_1995.tif reprojected.
Barrow_1995.tif masked.
Barrow_1995_01avg_aq.tif saved.
Barrow_1995.tif removed.
Barrow_1999.tif reprojected.
Barrow_1999.tif masked.
Barrow_1999_02avg_aq.tif saved.
Barrow_1999.tif removed.
Barrow_2000.tif reprojected.
Barrow_2000.tif masked.
Barrow_2000_09avg_aq.tif saved.
Barro

### 4.| Subpixel contours

In [26]:
%%time 
print(("--")*10,"Treating",tile_name,("--")*10)
folder_path = os.path.join(data_dir,tile_name)

# Create shorelines
shorelines_path = os.path.join(folder_path,tile_name+"_shorelines") 
if not os.path.exists(shorelines_path):
    shorelines = []
    print("Process shorelines...")    
    raster_paths = glob.glob(os.path.join(data_dir,tile_name,"*aq.tif"))
    raster_paths.sort()
    for r in raster_paths:
        # create path for single shorelines 
        sl_folder_path = os.path.join(folder_path,tile_name+"_single_shorelines")
        sl_path = os.path.join(sl_folder_path,os.path.splitext(os.path.basename(r))[0]+"_shoreline")
        # save single shoreline without modifications as backup
        if not os.path.exists(sl_path):
            with rio.open(r,"r") as raster:
                mndwi = raster.read(1)
                if np.count_nonzero(mndwi) > 0 and np.count_nonzero(~np.isnan(mndwi)) > 0:                    
                    thres = threshold_otsu(mndwi[~np.isnan(mndwi)])
                    print(thres)
                    shoreline = postprocess.subpixel_contours(r,thres)
                    if not shoreline.empty:
                        if not os.path.exists(sl_folder_path): os.mkdir(sl_folder_path)
                        shoreline.to_file(os.path.join(sl_path),driver="GeoJSON")
        else:
            shoreline = gpd.read_file(sl_path)
        # postprocess raw shorelines
        shoreline = gpd.clip(shoreline,buffer)
        cleaned = postprocess.remove_small_lines(shoreline, min_size=min_length)
        if not cleaned.empty:
            year = os.path.basename(r)[7:11]
            avg_aq = os.path.basename(r)[12:14]
            cleaned['id']=year
            cleaned = cleaned.dissolve(by=cleaned.id,aggfunc="sum")
            cleaned['year']=year
            cleaned['avg_aq']=avg_aq
            cleaned['otsu_thres']=str(thres)
            shorelines.append(cleaned)
            print(year+": shoreline processed.")
    shorelines_gdf = pd.concat(shorelines,ignore_index=True)    
    shorelines_gdf.to_file(os.path.join(shorelines_path),driver="GPKG")
    print("All shorelines have been created and saved.")
else:
    print("Shorelines already exist.")

-------------------- Treating Barrow --------------------
Process shorelines...
0.21091056
1985: shoreline processed.
0.46515965
1986: shoreline processed.
1990: shoreline processed.
0.1778094
1995: shoreline processed.
0.22073612
1999: shoreline processed.
0.38250306
2000: shoreline processed.
0.8308344
2001: shoreline processed.
0.8407794
2002: shoreline processed.
0.24962574
2003: shoreline processed.
0.87164164
2004: shoreline processed.
0.3492409
2005: shoreline processed.
0.31950837
2006: shoreline processed.
0.31681895
2007: shoreline processed.
0.34584063
2008: shoreline processed.
0.41513437
2009: shoreline processed.
0.43543744
2010: shoreline processed.
0.86784446
2011: shoreline processed.
0.2945012
2012: shoreline processed.
0.3619105
2013: shoreline processed.
0.37599027
2014: shoreline processed.
0.30046397
2015: shoreline processed.
0.55121434
2016: shoreline processed.
0.51456934
2017: shoreline processed.
0.8831848
2018: shoreline processed.
0.4468025
2019: shoreline 

### 5.| Create minimum water extent polygon

Generate minimum and maximum water extent raster for each processing tile

In [32]:
# Calculate raster with min and max water extent 
print(("--")*10,"Treating",tile_name,("--")*10)
folder_path = os.path.join(data_dir,tile_name)

if os.path.exists(folder_path):
    raster_paths = glob.glob(os.path.join(folder_path,"*aq.tif"))
    # make MNDWI images binary first
    # (this step should later be included to the shoreline extraction script, where the Otsu is already being calculated) 
    for r in raster_paths:
        # create binary raster using the Otsu threshold for min water raster                
        binary_file = os.path.join(folder_path,os.path.splitext(os.path.basename(r))[0]+"_bin.tif")
        if not os.path.exists(binary_file):
            with rio.open(r,"r") as raster:
                mndwi = raster.read(1)
                if np.count_nonzero(mndwi) > 0 and np.count_nonzero(~np.isnan(mndwi)) > 0:                    
                    meta = raster.meta
                    thres = threshold_otsu(mndwi[~np.isnan(mndwi)])
                    binary = mndwi.copy()
                    binary[binary > thres] = 1
                    binary[binary < thres] = 0
                    meta.update({
                        "compress":"LZW",
                        })
                    with rio.open(binary_file,'w',**meta) as dst:
                        dst.write(binary,1)
                    print(binary_file, "saved.")
        else:
            print(binary_file, "exists.")
    binary_paths = glob.glob(os.path.join(folder_path,"*aq_bin.tif"))
    min_water_file = os.path.join(data_dir,tile_name,tile_name+"_min_water_extent")
    max_water_file = os.path.join(data_dir,tile_name,tile_name+"_max_water_extent")
    if not os.path.exists(min_water_file):
        analysis.calc_water_extent(binary_paths,min_water_file,max_water_file)
    else:
        print("Files exist.")

-------------------- Treating Barrow --------------------
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2016_13avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2017_15avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2000_09avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2008_12avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_1986_04avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2018_17avg_aq_bin.tif saved.
/Users/Ronjamac/Documents/02_Studium/Masterarbeit/Code/VN_coastline_dynamics/data/Alaska_north/Barrow/Barrow_2014_

Generalize, vectorize and merge minimum water extent rasters and create transects

In [34]:
%%time
#remove small pixel cluster in min and max water extent rasters and merge rasters of all tiles
print(("--")*10,"Treating",tile_name,("--")*10)
folder_path = os.path.join(data_dir,tile_name)

# define path for generalized min water extent raster 
min_water_simple_file = os.path.join(data_dir,tile_name,tile_name+"_min_water_extent_simple")
if not os.path.exists(min_water_simple_file+"_poly"):
    try:
        # read min water extent file
        min_water_file = os.path.join(folder_path,tile_name+"_min_water_extent")
    except FileNotFoundError:
        print('File does not exist.')
    else:
        # remove small objects from raster
        analysis.remove_pixel_cluster(min_water_file,min_water_simple_file,50000,100000,0)
        print("Pixel cluster removed.")
        # vectorize raster 
        min_water_poly = analysis.vectorize_raster(min_water_simple_file,0)
        if not min_water_poly.empty:
            min_water_poly.to_file(min_water_simple_file+"_poly",driver="GeoJSON")
            print("Minimum water extent polygon created.\n")
else:
    print("Minimum water extent polygon already exists.\n")


-------------------- Treating Barrow --------------------
Pixel cluster removed.
Minimum water extent polygon created.

CPU times: user 28 s, sys: 7.85 s, total: 35.9 s
Wall time: 46.7 s


'    \n    min_water_poly = gpd.read_file(os.path.join(folder_path,tile_name+"_min_water_extent_simple_poly"))\n    min_water_polys.append(min_water_poly)\n# Concatenate all polygons\nmin_water_polys_gdf = pd.concat(min_water_polys,ignore_index=True)\n# Dissolve overlapping polygons\ngeoms = min_water_polys_gdf.geometry.unary_union\nmin_water_polys_gdf = gpd.GeoDataFrame(geometry=[geoms],crs=crs)\nmin_water_polys_gdf = min_water_polys_gdf.explode().reset_index(drop=True)\nmin_water_polys_gdf.to_file(all_min_water_polys_file,driver="GeoJSON")\nprint("Minimum water extent raster for Vietnam has been saved.")'

### 6.| Create transects

In [65]:
country_bounds = gpd.read_file(country_bounds_path).to_crs(crs)
country_bounds.geometry = country_bounds.buffer(-6500)

try:
    transects = gpd.read_file(transects_path)
    print("Transects exist and have been loaded.")
except:
    print("Create transects...")
    country_bounds = country_bounds.explode()
    # only take the land polygon to exclude islands etc.
    country_bounds['area'] = country_bounds.geometry.area
    country_bounds = country_bounds[country_bounds.area == np.max(country_bounds.area)]
    #country_bounds_gov.geometry = country_bounds_gov.geometry.simplify(500,preserve_topology=True)
    country_bounds.to_file(country_bounds_path+"_simple",driver="GeoJSON")
    # draw transects at country polygon 
    transects = postprocess.draw_transects_polygon(
        country_bounds,
        transect_len/2,
        transect_len/2,
        transect_dist,
        transect_min_line_length,
        sigma=3,
        out_path_poly=country_bounds_path+"_smooth"
        )
    # clip transects to buffer 
    transects = gpd.clip(transects,buffer)
    transects = transects.dropna() # if transects have been created along a multipolygon
    transects = transects.explode().reset_index(drop=True)
    transects.to_file(transects_path,driver="GeoJSON")
    print("Transects have been created and saved.")
else:
    # clip transects to min water extent raster
    if not os.path.exists(transects_path+"_clip"):
        print("Clip transects to min water extent...")
        min_water_buffer = min_water_poly.buffer(100)
        transects_clip = gpd.clip(transects,min_water_buffer)
        # convert all mutlilinestrings to single linestrings to treat transect pieces separately
        transects_clip = transects_clip.explode().reset_index()
        transects_clip.to_file(transects_path+"_clip",driver="GPKG")
        print("Transects haven been clipped to min water extent polygon and saved.")
    else:
        print("Clipped transects already exist.")

Create transects...
(0, 0) n_points: 13477
Transects have been created and saved.


### 7.| Calculate intersections between shorelines and transects

In [66]:
# Calculate intersections
# Load transects 
try:
    transects = gpd.read_file(transects_path)
    print("Transects have been loaded.")
except FileNotFoundError:
    print("Transects file does not exist.")

# Intersections
print(("--")*10,"Treating",tile_name,("--")*10)
folder_path = os.path.join(data_dir,tile_name)
intersections_file = os.path.join(data_dir,tile_name,tile_name+"_intersections")
if not os.path.exists(intersections_file):
    try:
        shorelines = gpd.read_file(os.path.join(data_dir,tile_name,tile_name+"_shorelines"))
    except FileNotFoundError:
        print('Shorelines do not exist.')
    else:        
        print("Calcualte intersections...")
        #tile_poly = tile.geometry
        #transects = gpd.clip(transects,tile_poly)
        intersections = postprocess.compute_intersections(transects,shorelines,remove_outliers=False)
        if not intersections.empty:
            intersections.to_file(intersections_file,driver="GPKG")
            print("Intersections have been created and saved.")
        else:
            print("No intersections available for",tile_name)
else:
    print("Intersections already exist.")

662 intersected
5663 intersected
5664 intersected
5665 intersected
5666 intersected
5667 intersected
5668 intersected
5669 intersected
5670 intersected
5671 intersected
5672 intersected
5673 intersected
5674 intersected
5675 intersected
5676 intersected
5677 intersected
5678 intersected
5679 intersected
5680 intersected
5681 intersected
5682 intersected
5683 intersected
5684 intersected
5685 intersected
5686 intersected
5687 intersected
5688 intersected
5689 intersected
5690 intersected
5691 intersected
5692 intersected
5693 intersected
5694 intersected
5695 intersected
5696 intersected
5697 intersected
5698 intersected
5699 intersected
5700 intersected
5701 intersected
5702 intersected
5703 intersected
5704 intersected
5705 intersected
5706 intersected
5707 intersected
5708 intersected
5709 intersected
5710 intersected
5711 intersected
5712 intersected
5713 intersected
5714 intersected
5715 intersected
5716 intersected
5717 intersected
5718 intersected
5719 intersected
5720 intersecte

### 8.| Coastline Change Analysis

#### Hotspot analysis

In [72]:
# Create classification transects (Erosion, Accretion, Stable, etc.)
import warnings
warnings.filterwarnings('ignore')
classifications_path = os.path.join(data_dir,"AKn_all_classifications")
if not os.path.exists(classifications_path):
    intersections = gpd.read_file(os.path.join(data_dir,tile_name,tile_name+"_intersections"))
    classifications = postprocess.calc_change_metrics(intersections,2)
    classifications.to_file(classifications_path,driver="GPKG")
    print("Classifications saved.")
else:
    print("Classifications exist.")

Transect 3117 removed.
Transect 3121 removed.
Transect 3122 removed.
Transect 3184 removed.
Transect 3185 removed.
Transect 3186 removed.
Transect 3204 removed.
Transect 3233 removed.
Transect 3259 removed.
Transect 3283 removed.
Transect 3284 removed.
Transect 3285 removed.
Transect 3309 removed.
Transect 3310 removed.
Transect 3311 removed.
Transect 3312 removed.
Transect 3313 removed.
Transect 3314 removed.
Transect 3315 removed.
Transect 3316 removed.
Transect 3341 removed.
Transect 3354 removed.
Transect 3355 removed.
Transect 3357 removed.
Transect 3358 removed.
Transect 3359 removed.
Transect 3360 removed.
Transect 3361 removed.
Transect 3362 removed.
Transect 3363 removed.
Transect 3374 removed.
Transect 3430 removed.
Transect 3431 removed.
Transect 3432 removed.
Transect 3433 removed.
Transect 3434 removed.
Transect 3511 removed.
Transect 3512 removed.
Transect 3513 removed.
Transect 3517 removed.
Transect 3762 removed.
Transect 3763 removed.
Transect 3764 removed.
Transect 37

In [5]:
### THIS HAS NOT BEEN DONE FOR ALASKA YET. 

# Calculate accretion and erosion hotspots 
erosion_hotspots_path = os.path.join(data_dir,"VN_erosion_hotspots")
accretion_hotspots_path = os.path.join(data_dir,"VN_accretion_hotspots")

if not os.path.exists(erosion_hotspots_path+"a"):
    # load an prepare classification file 
    classification = gpd.read_file(os.path.join(data_dir,"VN_all_classifications"))
    classification = classification.sort_values(by="Transect_id").reset_index(drop=True)
    #classification = classification.replace("nan",np.NaN)
    classification = classification[classification["class_L1"].notna()].reset_index(drop=True)

    # set up while loop to find clustered erosion and accretion classification transects
    yet_seen = []
    erosion_hotspots = []
    accretion_hotspots = []
    # iterate through all classification transects
    for t, transect in classification.iterrows():
            # check if transect has already been evaluated 
            if not t in yet_seen:
                cluster = []
                # create cluster if the next transect or the transect after has the same class 
                while (transect.class_L1 == classification.iloc[t].class_L1) or (transect.class_L1 == classification.iloc[t+1].class_L1) :
                    yet_seen.append(t)
                    cluster.append(classification.iloc[t])
                    t +=1
                    # exist while loop if last transect id +2 has been reached
                    if t+2 > len(classification):
                        break
                # if less than 20 transect share the same class, omit from hotspot analysis
                if len(cluster)>20:
                    # save only erosion and accretion clusters and convert to GeoDataFrame
                    if cluster[0].class_L1 == "Accretion":
                        gdf = gpd.GeoDataFrame(cluster)
                        accretion_hotspots.append(gdf)
                    elif cluster[0].class_L1 == "Erosion":
                        gdf = gpd.GeoDataFrame(cluster)
                        erosion_hotspots.append(gdf)
    print("All hotspots identified.")
    # add cluster number to hotspot transects
    for i, transect in enumerate(erosion_hotspots):
        transect['cluster_no'] = i
    for i, transect in enumerate(accretion_hotspots):
        transect['cluster_no'] = i 
    # merge all hotspot dataframes
    erosion_hotspots_gdf = pd.concat(erosion_hotspots)
    accretion_hotspots_gdf = pd.concat(accretion_hotspots)
    # save dataframes
    erosion_hotspots_gdf.to_file(erosion_hotspots_path,driver="GPKG")
    accretion_hotspots_gdf.to_file(accretion_hotspots_path,driver="GPKG")
    print("Hotspot files saved.")
else:
    print("Hotspot files exist.")

All hotspots identified.
Hotspot files saved.


In [9]:
# Filter extreme erosion and accretion hotspots
extreme_erosion_hotspots_path = os.path.join(data_dir,"VN_extreme_erosion_hotspots")
extreme_accretion_hotspots_path = os.path.join(data_dir,"VN_extreme_accretion_hotspots")
if not os.path.exists(extreme_erosion_hotspots_path):
    erosion_hotspots = gpd.read_file(os.path.join(data_dir,"VN_erosion_hotspots"))
    accretion_hotspots = gpd.read_file(os.path.join(data_dir,"VN_accretion_hotspots"))
    print("Mean erosion of extreme hotspots:")
    extreme_erosion_hotspots = postprocess.define_severe_hotspots(erosion_hotspots,-5,"smaller")
    print("Mean accretion of extreme hotspots:")
    extreme_accretion_hotspots = postprocess.define_severe_hotspots(accretion_hotspots,5,"bigger")
    # save files
    extreme_erosion_hotspots.to_file(extreme_erosion_hotspots_path,driver="GPKG")
    extreme_accretion_hotspots.to_file(extreme_accretion_hotspots_path,driver="GPKG")
else:
    print("Extreme hotspot files exist.")

Extreme hotspot files exist.


#### Land area change

In [10]:
# calcualte land area change in the coastal zone (5km buffer) as proportional change 
provinces_clip_path = os.path.join(data_dir,"VN_coastal_provinces_clip")
provinces_path = os.path.join(data_dir,"VN_coastal_provinces")
buffer_path = os.path.join(data_dir,"VN_buffer_5km")

# buffer coastal provinces by 5km and clip to buffer 
if not os.path.exists(provinces_clip_path):
    buffer = gpd.read_file(buffer_path)
    provinces = gpd.read_file(provinces_path)
    provinces = provinces.to_crs(crs)
    provinces['geometry'] = provinces.geometry.buffer(5000)
    provinces = gpd.clip(provinces,buffer)
    provinces.to_file(provinces_clip_path,driver="GeoJSON")
else:
    print("Coastal provinces are already clipped to buffer and read.")
    provinces = gpd.read_file(provinces_clip_path)

# calcualte proportional land area change for each province 
for i in provinces.index:
    name = provinces.ADM1_PCODE.iloc[i]
    print("Land area change in province: ",name)
    land_area_path = os.path.join(plot_dir,"land_area_change_"+name)
    if not os.path.exists(land_area_path):
        land_area = pd.DataFrame(columns=['year','land_area_percentage'])
        land_area['year'] = range(1987,2022)
        for year in range(1987,2022):
            print(year)
            binary_files = glob.glob(os.path.join("{path}/**/*{year}*bin.tif".format(path=data_dir,year=year)))
            binary_files.sort()
            land_pixels, valid_pixels = [],[]
            for file in binary_files:
                # crop raster to area
                area = provinces[provinces.index==i]
                with rio.open(file) as src:
                    try:
                        out_image, out_transform = rio.mask.mask(src,area.geometry,crop=True,nodata=np.nan)
                        out_meta = src.meta
                        out_meta.update({"driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform,
                            "compress":"LZW"})
                        out_path = os.path.join(os.getcwd(),"test_data",str(year)+"_"+name+"_"+os.path.basename(file)[:3])
                        with rio.open(out_path,"w",**out_meta) as dst:
                            dst.write(out_image)
                        show(out_image)
                        im_rev = out_image.copy()
                        im_rev[im_rev==0]=2
                        im_rev[im_rev==1]=0
                        im_rev[im_rev==2]=1
                        valid_pixel = np.count_nonzero(~np.isnan(out_image))
                        land_pixel = np.nansum(im_rev)#*30*30
                        land_pixels.append(land_pixel)
                        valid_pixels.append(valid_pixel)
                    except ValueError:
                        pass
            land_pixel_percentage = np.sum(land_pixels)/np.sum(valid_pixels)*100
            print("\nLand pixel percentage in",str(year)+":", land_pixel_percentage,"\n")
            land_area.loc[land_area.year == year, "land_area_percentage"] = land_pixel_percentage
            land_area.to_csv(land_area_name)
    else:
        print("Land area file already exists.")

Coastal provinces are already clipped to buffer and read.
Land area change in province:  VN717
Land area file already exists.
Land area change in province:  VN821
Land area file already exists.
Land area change in province:  VN811
Land area file already exists.
Land area change in province:  VN507
Land area file already exists.
Land area change in province:  VN715
Land area file already exists.
Land area change in province:  VN823
Land area file already exists.
Land area change in province:  VN501
Land area file already exists.
Land area change in province:  VN713
Land area file already exists.
Land area change in province:  VN405
Land area file already exists.
Land area change in province:  VN103
Land area file already exists.
Land area change in province:  VN701
Land area file already exists.
Land area change in province:  VN511
Land area file already exists.
Land area change in province:  VN813
Land area file already exists.
Land area change in province:  VN113
Land area file alread